You have to work on the [Dogs adoptions](https://drive.google.com/file/d/1wQsA0oB6wwYlnkvvcyBCmLk7QmgVWNax/view?usp=sharing) dataset. 

It contains three files:
*  `dogs.csv`, shortly *dogs*
*  `dogTravel.csv`, shortly *travels*
*  `NST-EST2021-POP.csv`

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).
1.    At most 3 students can be in each group. You must create the groups by yourself.
1.    You do not have to send me the project *before* the discussion.

In [1]:
import pandas as pd 
import re 
#To resolve a warning related to pandas object, we type follow instruction
#pd.options.mode.chained_assignment = None  # default='warn'

import warnings
warnings.filterwarnings('ignore')

In [2]:
#import dogs dataset 
dogs=pd.read_csv('dogs.csv')
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


### Explain problems in data set "Dogs"

In [3]:
#Looking at the dataset, we can see that there are some problems in a few lines. 
#In particular, there are lines in which the attributes are shifted by one value. To explain better, 
#the attribute "posted" is related to the timestamp but when
pd.to_datetime(dogs['posted']) 
#is performed, there is an error cause in some rows the value of "posted" is not a date, but a name of a city. 
#Analysing this problem more, we saw that in some lines, the value of posted is related to "contact_city" and so on for few columns. 
#As a result, we should shift back some values related to columns from "status" to "accessed". 

ParserError: Unknown string format: Las Vegas

In [4]:
indexes= dogs[dogs['accessed'].isnull()].index
indexes=list(indexes)
ind_prob=dogs[dogs['accessed'].isnull()]
ind_prob['accessed']=ind_prob['stateQ']
ind_prob['stateQ']=ind_prob['contact_country']
ind_prob['contact_country']=ind_prob['contact_zip']
ind_prob['contact_zip']=ind_prob['contact_state']
ind_prob['contact_state']=ind_prob['contact_city']
ind_prob['contact_city']=ind_prob['posted']
ind_prob['posted']=ind_prob['status']
ind_prob['status']='NULL'

In [5]:
dogs_1=dogs.drop(labels=indexes, axis=0)
dogs_new=pd.concat([dogs_1,ind_prob], ignore_index=True)
dogs_new
#dogs_new is the new data set with corrections

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,45916348,WA581,https://www.petfinder.com/dog/cody-45916348/wa...,Dog,Dog,Golden Retriever,Terrier,True,False,Bicolor,...,Seattle,WA,98106,US,WA,2019-09-20,Dog,*Please apply online at www.6dogrees.com to ge...,92,164.55
58176,45733027,WA581,https://www.petfinder.com/dog/gracie-45733027/...,Dog,Dog,Papillon,Cavalier King Charles Spaniel,True,False,Golden,...,Seattle,WA,98106,US,WA,2019-09-20,Dog,"Meet \""Gracie\"" a very beautiful 11 pound,fema...",82,168.54
58177,45413997,WA581,https://www.petfinder.com/dog/jameson-45413997...,Dog,Dog,Rat Terrier,Chihuahua,True,False,"Tricolor (Brown, Black, & White)",...,Seattle,WA,98106,US,WA,2019-09-20,Dog,"Meet \""Jameson\"" He is a very handsome 9 pound...",108,297.27
58178,45406516,WA581,https://www.petfinder.com/dog/canelo-45406516/...,Dog,Dog,Chihuahua,Terrier,True,False,Bicolor,...,Seattle,WA,98106,US,WA,2019-09-20,Dog,"Meet Canelo a adorable small, tan very sweet...",94,312.10


### 1. Extract all dogs with status that is not *adoptable*

In [6]:
dogs_new[dogs_new['status']!='adoptable']

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
58147,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
58148,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
58149,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
58150,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
58151,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
58152,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
58153,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
58154,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,Bristow,VA,20136,US,DC,2019-09-20,Dog,This 10-year young senior is very sweet and lo...,76,263.63
58155,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,Silver Spring,MD,20905,US,DC,2019-09-20,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
58156,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,Gettysburg,PA,17325,US,DC,2019-09-20,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


### 2. For each (primary) breed, determine the number of dogs

In [7]:
dogs_new[['breed_primary', 'id']].groupby('breed_primary').count()

,id
breed_primary,
Affenpinscher,17
Afghan Hound,4
Airedale Terrier,19
Akbash,3
Akita,181
...,...
Wirehaired Pointing Griffon,1
Wirehaired Terrier,60
Xoloitzcuintli / Mexican Hairless,11


### 3. For each (primary) breed, determine the ratio between the number of dogs of `Mixed Breed` and those not of Mixed Breed. Hint: look at the `secondary_breed`.

In [8]:
#In secondary_breed for a number of dogs there is Mixed breed, we have to find them 
dogs_new['y_mix']=dogs_new['breed_secondary']=='Mixed Breed'
dogs_new['n_mix']=dogs_new['breed_secondary']!='Mixed Breed'
ratio= dogs_new[['breed_primary', 'y_mix', 'n_mix']].groupby('breed_primary').sum() 
ratio['ratio']=ratio['y_mix']/ratio['n_mix']
ratio

,y_mix,n_mix,ratio
breed_primary,,,
Affenpinscher,1,16,0.062500
Afghan Hound,0,4,0.000000
Airedale Terrier,1,18,0.055556
Akbash,0,3,0.000000
Akita,6,175,0.034286
...,...,...,...
Wirehaired Pointing Griffon,0,1,0.000000
Wirehaired Terrier,4,56,0.071429
Xoloitzcuintli / Mexican Hairless,0,11,0.000000


### 4. For each (primary) breed, determine the earliest and the latest `posted` timestamp.

In [9]:
dogs_new['posted']

0        2019-09-20T16:37:59+0000
1        2019-09-20T16:24:57+0000
2        2019-09-20T14:10:11+0000
3        2019-09-20T10:08:22+0000
4        2019-09-20T06:48:30+0000
                   ...           
58175    2019-09-09T17:05:17+0000
58176    2019-08-24T16:32:31+0000
58177    2019-07-31T16:21:07+0000
58178    2019-07-29T21:50:16+0000
58179    2019-07-15T16:57:05+0000
Name: posted, Length: 58180, dtype: object

In [10]:
dogs_new['posted_date']=pd.to_datetime(dogs_new['posted']) 
a= dogs_new[['breed_primary', 'posted_date']].groupby('breed_primary').min()
b= dogs_new[['breed_primary', 'posted_date']].groupby('breed_primary').max()
pd.merge(a,b,on='breed_primary', suffixes=('_earliest','_latest'))

,posted_date_earliest,posted_date_latest
breed_primary,,
Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00
...,...,...
Wirehaired Pointing Griffon,2016-06-29 20:03:55+00:00,2016-06-29 20:03:55+00:00
Wirehaired Terrier,2012-11-27 14:07:54+00:00,2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00+00:00,2019-09-08 11:15:54+00:00


### 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [11]:
dogs_new['fem']=dogs_new['sex']=='Female'
dogs_new['male']=dogs_new['sex']=='Male'
sex_imb = dogs_new[['contact_state', 'male', 'fem']].groupby('contact_state').sum()
sex_imb['difference']=sex_imb['male']-sex_imb['fem']
sex_imb[sex_imb['difference']==sex_imb['difference'].max()]
#OH is the state where the imbalance is largest

,male,fem,difference
contact_state,,,
OH,1439,1234,205


### 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [12]:
dogs_new[['age','size','stay_duration', 'stay_cost']].groupby(['age','size']).mean()

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

### 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [13]:
travel= pd.read_csv('dogTravel.csv', index_col='index')
n_trav=travel[['id','contact_city']].groupby('id', as_index=False).count()
n_trav=n_trav.rename(columns={"contact_city": "number_trav"})
trav_dogs=pd.merge(dogs_new[['id','breed_primary','breed_secondary','breed_mixed']],n_trav, on='id')
trav_dogs[trav_dogs['number_trav']>=3]

,id,breed_primary,breed_secondary,breed_mixed,number_trav
68,45642530,Jindo,NaN,False,4
236,46039420,Border Collie,Retriever,True,4
318,40036107,Pit Bull Terrier,NaN,True,3
578,45851842,Labrador Retriever,NaN,False,3
596,45841145,Mixed Breed,NaN,True,3
...,...,...,...,...,...
4011,38664932,Pit Bull Terrier,NaN,False,3
4012,38495992,Pit Bull Terrier,NaN,False,3
4083,45588395,German Shepherd Dog,NaN,True,3
4112,36978896,Alaskan Malamute,NaN,False,7


### 8. Fix the `travels` table so that the correct state is computed from  the `manual` and the `found` fields. If `manual` is not missing, then it overrides what is stored in `found`.

In [14]:
travel

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN


In [15]:
indices=travel[travel['manual'].notnull()].index
travel['found'][indices]=travel['manual'][indices]
travel

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN


### 9. For each state, compute the ratio between the number of travels and the population.

In [16]:
state_pop= pd.read_csv('NST-EST2021-POP.csv', names=('state', 'population'))
state_pop

,state,population
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223
5,Colorado,5.773.714
6,Connecticut,3.605.944
7,Delaware,989.948
8,District of Columbia,689.545
9,Florida,21.538.187


In [17]:
n_trav= travel[['found', 'id']].groupby('found').count()
new=pd.merge(n_trav, state_pop, left_index=True, right_on='state')
new['population']=new['population'].str.replace('\.','')
new['population']=new['population'].astype(int)
new= new.rename(columns={"id": "n_travels"})
new['ratio']=new['n_travels']/new['population']
new.set_index('state', inplace=True)
new

,n_travels,population,ratio
state,,,
Alabama,338,5024279,6.727333e-05
Arizona,33,7151502,4.614415e-06
Arkansas,61,3011524,2.025553e-05
California,279,39538223,7.056463e-06
Colorado,11,5773714,1.905186e-06
Connecticut,16,3605944,4.437118e-06
Delaware,27,989948,2.727416e-05
Florida,163,21538187,7.567954e-06
Georgia,235,10711908,2.193820e-05


### 10. For each dog, compute the number of days from the `posted` day to the day of last access.

In [18]:
dogs_new['accessed_date']=pd.to_datetime(dogs_new['accessed']) 
dogs_new['posted_date_true']=dogs_new['posted_date'].dt.date
dogs_new['posted_date_true']=pd.to_datetime(dogs_new['posted_date_true']) 
dogs_new['diff']=dogs_new['accessed_date']-dogs_new['posted_date_true']
dogs_new

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,stay_duration,stay_cost,y_mix,n_mix,posted_date,fem,male,accessed_date,posted_date_true,diff
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,70,124.81,True,False,2019-09-20 16:37:59+00:00,False,True,2019-09-20,2019-09-20,0 days
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,49,122.07,True,False,2019-09-20 16:24:57+00:00,False,True,2019-09-20,2019-09-20,0 days
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,87,281.51,False,True,2019-09-20 14:10:11+00:00,False,True,2019-09-20,2019-09-20,0 days
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,62,145.83,False,True,2019-09-20 10:08:22+00:00,True,False,2019-09-20,2019-09-20,0 days
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,93,241.09,False,True,2019-09-20 06:48:30+00:00,False,True,2019-09-20,2019-09-20,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,45916348,WA581,https://www.petfinder.com/dog/cody-45916348/wa...,Dog,Dog,Golden Retriever,Terrier,True,False,Bicolor,...,92,164.55,False,True,2019-09-09 17:05:17+00:00,False,True,2019-09-20,2019-09-09,11 days
58176,45733027,WA581,https://www.petfinder.com/dog/gracie-45733027/...,Dog,Dog,Papillon,Cavalier King Charles Spaniel,True,False,Golden,...,82,168.54,False,True,2019-08-24 16:32:31+00:00,True,False,2019-09-20,2019-08-24,27 days
58177,45413997,WA581,https://www.petfinder.com/dog/jameson-45413997...,Dog,Dog,Rat Terrier,Chihuahua,True,False,"Tricolor (Brown, Black, & White)",...,108,297.27,False,True,2019-07-31 16:21:07+00:00,False,True,2019-09-20,2019-07-31,51 days
58178,45406516,WA581,https://www.petfinder.com/dog/canelo-45406516/...,Dog,Dog,Chihuahua,Terrier,True,False,Bicolor,...,94,312.10,False,True,2019-07-29 21:50:16+00:00,False,True,2019-09-20,2019-07-29,53 days


### 11. Partition the dogs according to the number of weeks from the `posted` day to the day of last access.

In [19]:
dogs_new['weeks']=dogs_new['diff']//7
dogs_new

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,stay_cost,y_mix,n_mix,posted_date,fem,male,accessed_date,posted_date_true,diff,weeks
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,124.81,True,False,2019-09-20 16:37:59+00:00,False,True,2019-09-20,2019-09-20,0 days,0 days 00:00:00
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,122.07,True,False,2019-09-20 16:24:57+00:00,False,True,2019-09-20,2019-09-20,0 days,0 days 00:00:00
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,281.51,False,True,2019-09-20 14:10:11+00:00,False,True,2019-09-20,2019-09-20,0 days,0 days 00:00:00
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,145.83,False,True,2019-09-20 10:08:22+00:00,True,False,2019-09-20,2019-09-20,0 days,0 days 00:00:00
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,241.09,False,True,2019-09-20 06:48:30+00:00,False,True,2019-09-20,2019-09-20,0 days,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,45916348,WA581,https://www.petfinder.com/dog/cody-45916348/wa...,Dog,Dog,Golden Retriever,Terrier,True,False,Bicolor,...,164.55,False,True,2019-09-09 17:05:17+00:00,False,True,2019-09-20,2019-09-09,11 days,1 days 13:42:51.428571428
58176,45733027,WA581,https://www.petfinder.com/dog/gracie-45733027/...,Dog,Dog,Papillon,Cavalier King Charles Spaniel,True,False,Golden,...,168.54,False,True,2019-08-24 16:32:31+00:00,True,False,2019-09-20,2019-08-24,27 days,3 days 20:34:17.142857142
58177,45413997,WA581,https://www.petfinder.com/dog/jameson-45413997...,Dog,Dog,Rat Terrier,Chihuahua,True,False,"Tricolor (Brown, Black, & White)",...,297.27,False,True,2019-07-31 16:21:07+00:00,False,True,2019-09-20,2019-07-31,51 days,7 days 06:51:25.714285714
58178,45406516,WA581,https://www.petfinder.com/dog/canelo-45406516/...,Dog,Dog,Chihuahua,Terrier,True,False,Bicolor,...,312.10,False,True,2019-07-29 21:50:16+00:00,False,True,2019-09-20,2019-07-29,53 days,7 days 13:42:51.428571428


In [20]:
dogs_new['weeks']=dogs_new['weeks'].astype(str)
dogs_new['weeks']=dogs_new['weeks'].str.extract(r'^(\d+)')
dogs_new['weeks']=dogs_new['weeks'].astype(int)
dogs_new['weeks']

0        0
1        0
2        0
3        0
4        0
        ..
58175    1
58176    3
58177    7
58178    7
58179    9
Name: weeks, Length: 58180, dtype: int32

In [21]:
partition= dogs_new[['id','weeks']].groupby('weeks').count()
partition=partition.rename(columns={'id':'Number_dogs'})
partition.sort_index()

,Number_dogs
weeks,
0,9803
1,6547
2,5764
3,3353
4,2439
...,...
729,1
746,1
811,1


### 12. Find for duplicates in the `dogs` dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [22]:
#cond1
no_null=dogs_new[dogs_new['description'].notna()]
no_null
a=no_null[['breed_primary', 'breed_secondary', 'sex', 'id']].groupby(['breed_primary', 'breed_secondary','sex']).count()
a= a[a['id']!=1].reset_index()
a= a.rename(columns={"id": "number"})
a

,breed_primary,breed_secondary,sex,number
0,Akita,German Shepherd Dog,Female,2
1,Akita,German Shepherd Dog,Male,3
2,Akita,Hound,Female,2
3,Akita,Jindo,Male,2
4,Akita,Labrador Retriever,Female,3
...,...,...,...,...
1874,Yorkshire Terrier,Shih Tzu,Female,2
1875,Yorkshire Terrier,Shih Tzu,Male,8
1876,Yorkshire Terrier,Terrier,Female,7
1877,Yorkshire Terrier,Terrier,Male,5


In [23]:
#extract id
dicti={}
for j in range(len(a)):
    dicti[j]=no_null[(no_null['breed_primary']==a.iloc[j]['breed_primary']) & (no_null['breed_secondary']==a.iloc[j]['breed_secondary'])&(no_null['sex']==a.iloc[j]['sex'])]['id']
dicti

{0: 42436    46021163
 42437    46021089
 Name: id, dtype: int64,
 1: 39093    42994498
 39239    28662641
 48876    44589975
 Name: id, dtype: int64,
 2: 15025    36772830
 15133    34447648
 Name: id, dtype: int64,
 3: 17557    45591802
 17560    45591568
 Name: id, dtype: int64,
 4: 26626    45600507
 54711    45845428
 57816    45078019
 Name: id, dtype: int64,
 5: 898      44752747
 6832     46039421
 16327    45926727
 Name: id, dtype: int64,
 6: 1448     45713807
 12071    45479494
 Name: id, dtype: int64,
 7: 19011    43947972
 28782    45509063
 44457    42781633
 Name: id, dtype: int64,
 8: 37967    45923186
 39170    34650124
 Name: id, dtype: int64,
 9: 1265     46036163
 18978    44070616
 27999    42190659
 36987    46016972
 Name: id, dtype: int64,
 10: 42759    45564411
 58118    45813878
 58157    33218331
 Name: id, dtype: int64,
 11: 1512     45381147
 1515     45380618
 2954     46005852
 18815    44378530
 23752    45555559
 24904    37663412
 27468    44803893
 27

In [24]:
#put id in the dataset
for i in range(len(dicti)):
    dicti[i]=dicti[i].values.tolist()
a['id_list']=dicti.values()
a

,breed_primary,breed_secondary,sex,number,id_list
0,Akita,German Shepherd Dog,Female,2,"[46021163, 46021089]"
1,Akita,German Shepherd Dog,Male,3,"[42994498, 28662641, 44589975]"
2,Akita,Hound,Female,2,"[36772830, 34447648]"
3,Akita,Jindo,Male,2,"[45591802, 45591568]"
4,Akita,Labrador Retriever,Female,3,"[45600507, 45845428, 45078019]"
...,...,...,...,...,...
1874,Yorkshire Terrier,Shih Tzu,Female,2,"[45071933, 46036131]"
1875,Yorkshire Terrier,Shih Tzu,Male,8,"[45579559, 46033029, 45889344, 45889036, 45829..."
1876,Yorkshire Terrier,Terrier,Female,7,"[45908725, 36535340, 44404139, 44404138, 45018..."
1877,Yorkshire Terrier,Terrier,Male,5,"[43693991, 42612948, 45965556, 45397822, 45958..."


In [25]:
#from the dataset 'no_null', extract description associated to specific id in 
list_id = a['id_list']
b=[]
lista_def=[]
for i in range(len(list_id)):
    b=[]
    for k in range(len(list_id[i])):
        b.append(no_null[no_null['id']==list_id[i][k]]['description'].values[0])
    lista_def.append(b)

In [26]:
a['description']=lista_def
a

,breed_primary,breed_secondary,sex,number,id_list,description
0,Akita,German Shepherd Dog,Female,2,"[46021163, 46021089]","[Mika is a blend of Akita, Husky and Shepard. ..."
1,Akita,German Shepherd Dog,Male,3,"[42994498, 28662641, 44589975]",[A warrior in a small package. If you want to...
2,Akita,Hound,Female,2,"[36772830, 34447648]","[Please note, this is a courtesy listing for o..."
3,Akita,Jindo,Male,2,"[45591802, 45591568]",[Seon is 11 months and 27 lbs. M\n\nSeon may b...
4,Akita,Labrador Retriever,Female,3,"[45600507, 45845428, 45078019]",[Please contact Snap (adoptions@snap2it.org) f...
...,...,...,...,...,...,...
1874,Yorkshire Terrier,Shih Tzu,Female,2,"[45071933, 46036131]",[Meet Diana! This wheeled girl has been throug...
1875,Yorkshire Terrier,Shih Tzu,Male,8,"[45579559, 46033029, 45889344, 45889036, 45829...",[Scooter and Fudge are very sweet boys. They a...
1876,Yorkshire Terrier,Terrier,Female,7,"[45908725, 36535340, 44404139, 44404138, 45018...",[Meet Lucy! Lucy is a 10 month old Carin Terr...
1877,Yorkshire Terrier,Terrier,Male,5,"[43693991, 42612948, 45965556, 45397822, 45958...",[FELIX IS A HANDSOME LITTLE FELLOW! He is a te...


In [27]:
from nltk.cluster.util import cosine_distance
import numpy as np
def get_cosine_similarity(sentence1, sentence2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sentence1 = [w.lower() for w in sentence1]
    sentence2 = [w.lower() for w in sentence2]
    words_all = list(set(sentence1 + sentence2))

    array1 = [0] * len(words_all)
    array2 = [0] * len(words_all)

    for w in sentence1:
        if w in stopwords:
            continue
        array1[words_all.index(w)] += 1

    for w in sentence2:
        if w in stopwords:
            continue
        array2[words_all.index(w)] += 1
    return 1 - cosine_distance(array1, array2)




In [28]:
for i in range(len(list_id)):
    for j in range(len(list_id[i])):
        for k in range(len(list_id[i])-1,j,-1):
            x= a['description'][i][j]
            y= a['description'][i][k]
            if get_cosine_similarity(x,y, stopwords=None)>=0.9:
                S1=x
                S2=y
                Cos=get_cosine_similarity(x,y, stopwords=None)
print(f"\033[1mThe first sentence is \033[0m \n{x}\n\033[1mThe second sentence is \033[0m\n{y} \n\n\033[1mThe similarity value is equal to\033[0m {get_cosine_similarity(x,y,stopwords=None)} \n \n ")

The first sentence is  
Bam Bam is approximately a 1 year old Yorkshire Terrier/Wire Haired Terrier ready to find his loving forever home! 

Bam Bam is a sweet, & loving boy whom is a gorgeous little boy who loves everyone including kids, and other dogs.  His favorite thing to do is to play, & to be by your side or in your lap. 

Bam Bam is fixed, microchipped and vaccinated. He would be a great addition to any family.

Fill out an application today to set up a meet & greet with this sweet boy!!

Application, Reference check, Home check and adoption fee apply.

BRAT Rescue, Inc. at this time conducts adoptions in the Five (5) boroughs of NYC, Long Island, and Southern Westchester County.

BRAT RESCUE INC., is a 501c3 tax exempt organization that rescues dog's and feeds feral animals in Far Rockaway and neighboring towns in Queens and the five boroughs of NYC.
The second sentence is 
Buckeye came into our rescue from the owner.  There were issues they were unable to handle.  He was bein

In [29]:
import time 
start=time.time()
id_1=[]
id_2=[]
descr_1=[]
descr_2=[]
sim=[]
for i in range(len(list_id)):
    for j in range(len(list_id[i])):
        for k in range(len(list_id[i])-1,j,-1):
            x= a['description'][i][j]
            y= a['description'][i][k]
            if get_cosine_similarity(x,y, stopwords=None)>=0.9:
                id_1.append(a['id_list'][i][j])
                id_2.append(a['id_list'][i][k])
                descr_1.append(x)
                descr_2.append(y)
                sim.append(get_cosine_similarity(x,y, stopwords=None))
end=time.time()
t=end-start
final={'id_1':id_1, 'descr_1':descr_1, 'id_2':id_2, 'descr_2':descr_2, 'similarity':sim}
pd.DataFrame.from_dict(final)                

,id_1,descr_1,id_2,descr_2,similarity
0,46021163,"Mika is a blend of Akita, Husky and Shepard. S...",46021089,"Yoko is a sweet 10 week old Akita, Husky, Shep...",0.978872
1,42994498,A warrior in a small package. If you want to ...,44589975,OREO and his brother LUDY were found under a h...,0.920485
2,42994498,A warrior in a small package. If you want to ...,28662641,"This smart fellow has it all, good looks, a br...",0.937452
3,28662641,"This smart fellow has it all, good looks, a br...",44589975,OREO and his brother LUDY were found under a h...,0.973067
4,36772830,"Please note, this is a courtesy listing for ou...",34447648,Sadie is around 6-8 years old and she's a real...,0.983449
...,...,...,...,...,...
281428,45965556,"AT PEPPERMINT PIG ANIMAL RESCUE, WE UPDATE ALL...",45397822,Felix and his brother Oscar came to us from a ...,0.956428
281429,45397822,Felix and his brother Oscar came to us from a ...,45958435,For a quicker response please submit a online ...,0.971458
281430,44070750,Look at our Angel Pancho enjoying life his at ...,44822880,Buckeye came into our rescue from the owner. ...,0.972511
281431,44070750,Look at our Angel Pancho enjoying life his at ...,45407894,Bam Bam is approximately a 1 year old Yorkshir...,0.975867


In [30]:
#minutes
t/60

15.501316328843435